# Monodromy - Taylor Brysiewicz (V 1.0)

In [1]:
using HomotopyContinuation

For more information about monodromy and computing  monodromy groups using numerical algebraic geometry see 

*Numerical computation of Galois groups*, Jonathan Hauenstein, Jose Israel Rodriguez, and Frank Sottile. Foundations of Computational Mathematics, 18 (2018) 867–890 DOI 10.1007/s10208-017-9356-x 

https://arxiv.org/pdf/1605.07806.pdf



For more information about solve polynomial systems using monodromy see

*Solving polynomial systems via homotopy continuation and monodromy*
Tim Duff, Cvetelina Hill, Anders Jensen, Kisun Lee, Anton Leykin, and Jeff Sommars.
IMA Journal of Numerical Analysis, 2018

https://arxiv.org/pdf/1609.08722.pdf

## Branched Covers

As in Sascha's talk, we consider a family of $0$-dimensional polynomial systems parametrized by some variety $Q$ (let's just take $Q=\mathbb{C}^k$ for some $k$)
$$Z = \{(x,p) \in \mathbb{C}^n \times Q \mid F(x;p) =0\} \subset \mathbb{C}^n \times Q$$
$$\,\,\,\, \downarrow \pi $$
$$Q$$

Over *most* parameters, there are $d$ solutions.

The map $\pi$ is called a **branched cover** 

(technically this means $Z \xrightarrow{\pi} Q$ is a dominant map of irreducible varieties of the same dimension)

There is a variety $D \subsetneq Q$ called the **discriminant** (or branch locus) consisting of the parameter values giving solutions with multiplicity.

On $U = Q \backslash D$ (the regular values), the map $\pi:Z|_{\pi^{-1}(U)} \to U$ is a $d$-to-$1$ **covering space**.






<img src="coveringspace.png" width="500">


### Example

$$F(x;a,b) = x^2+ax+b$$


In [2]:
@var x a b
F = System([x^2 + a * x + b], parameters = [a, b])

System of length 1
 1 variables: x
 2 parameters: a, b

 b + a*x + x^2

**Remember**: $U$ is connected as a **real** manifold.

Consequence: There exist loops $\gamma: [0,1] \to U$ around $D$ and based at $p=\gamma(0)=\gamma(1) \in U$.

<img src="monodromy1b.png" width="500">

In [3]:
S₀=[[-1],[1]]
p₀=[0,-1]

2-element Array{Int64,1}:
  0
 -1

In [4]:
p₁ = [0, im]
p₂ = [0,1]
p₃ = [0,-im]

2-element Array{Complex{Int64},1}:
 0 + 0im
 0 - 1im

In [5]:
#p0->p1
S₁ = solve(F,S₀; start_parameters=p₀,target_parameters=p₁)
#p1->p2
S₂ = solve(F,S₁; start_parameters=p₁,target_parameters=p₂)
#p2->p3
S₃ = solve(F,S₂; start_parameters=p₂,target_parameters=p₃)
#p3->p0 again
S₀2 = solve(F,S₃; start_parameters=p₃,target_parameters=p₀)

Tracking 2 paths... 100%|███████████████████████████████| Time: 0:00:06
  # paths tracked:                  2
  # non-singular solutions (real):  2 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         2 (0)


Result with 2 solutions
• 2 paths tracked
• 2 non-singular solutions (2 real)
• random_seed: 0x2ca5764e


In [6]:
show(stdout,"text/plain",hcat(S₀,solutions(S₁),solutions(S₂),solutions(S₃),solutions(S₀2)))

2×5 Array{Array{Complex{Float64},1},2}:
 [-1.0+0.0im]  [-0.707107+0.707107im]  [0.0+1.0im]  [0.707107+0.707107im]   [1.0+0.0im]
 [1.0+0.0im]   [0.707107-0.707107im]   [0.0-1.0im]  [-0.707107-0.707107im]  [-1.0+0.0im]

Note that the solutions have permuted.
<img src="monodromy1b.png" width="500">

In [7]:
#This function automates the process above, given a parametrized polynomial system F, start solutions 
# Sols, and a loop γ which is just a sequence of parameter values starting and ending with the parameter
# value for Sols

function monodromy_loop(F,Sols,γ)
    S=Sols;
    P=γ[1]
    for NewP in γ[2:end]
        newS=solve(F,S; start_parameters=P, target_parameters=NewP)
        S=newS
        P=NewP
    end
    return S
end

monodromy_loop (generic function with 1 method)

In [8]:
#Now let's take the loop again and see that we return to the solutions [[-1],[1]] in order.
M=monodromy_loop(F,solutions(S₀2),[p₀,p₁,p₂,p₃,p₀]);
solutions(M)

2-element Array{Array{Complex{Float64},1},1}:
 [-1.0 + 0.0im]
 [1.0 + 0.0im]

### Solving Systems using Monodromy

In [9]:
#What if we only knew about the solution 1?
#  Then performing the monodromy loop *discovers* the solution -1
solutions(monodromy_loop(F,[1],[p₀,p₁,p₂,p₃,p₀]))

1-element Array{Array{Complex{Float64},1},1}:
 [-1.0 + 0.0im]

In [10]:
@var c[1:20]
f = sum([c[i]*x^(i-1) for i in 1:20])
F = System([f],parameters=c)

System of length 1
 1 variables: x
 20 parameters: c₁, c₂, c₃, c₄, c₅, c₆, c₇, c₈, c₉, c₁₀, c₁₁, c₁₂, c₁₃, c₁₄, c₁₅, c₁₆, c₁₇, c₁₈, c₁₉, c₂₀

 c₁ + x*c₂ + x^2*c₃ + x^3*c₄ + x^4*c₅ + x^5*c₆ + x^6*c₇ + x^7*c₈ + x^8*c₉ + x^9*c₁₀ + x^10*c₁₁ + x^11*c₁₂ + x^12*c₁₃ + x^13*c₁₄ + x^14*c₁₅ + x^15*c₁₆ + x^16*c₁₇ + x^17*c₁₈ + x^18*c₁₉ + x^19*c₂₀

We just need a start solution

In [11]:
#So we can force the start solution to be 0.923+0.2im
S=[0.923+0.2*im]
C = randn(ComplexF64,20)
evaluation = sum([C[i]*S[1]^(i-1) for i in 1:20])
C[1]-=evaluation
evaluation = sum([C[i]*S[1]^(i-1) for i in 1:20])

-1.1102230246251565e-16 + 1.3877787807814457e-16im

Now we have 1 solution to some degree 19 polynomial 

In [12]:
SolutionsFound=[S]
loop_num=0
while length(SolutionsFound)<19
    loop_num+=1
    newSolution=solutions(monodromy_loop(F,S,[C,5*randn(ComplexF64,20),3*randn(ComplexF64,20),C]))
    if findfirst(x->x[1]≈(newSolution[1][1]),SolutionsFound)==nothing
        push!(SolutionsFound,newSolution[1])
        println("We found a new solution (Total:",length(SolutionsFound),")")
        flush(stdout)
    end
end
println("Finished after ",loop_num, " loops")
    

We found a new solution (Total:2)
We found a new solution (Total:3)
We found a new solution (Total:4)
We found a new solution (Total:5)
We found a new solution (Total:6)
We found a new solution (Total:7)
We found a new solution (Total:8)
We found a new solution (Total:9)
We found a new solution (Total:10)
We found a new solution (Total:11)
We found a new solution (Total:12)
We found a new solution (Total:13)
We found a new solution (Total:14)
We found a new solution (Total:15)
We found a new solution (Total:16)
We found a new solution (Total:17)
We found a new solution (Total:18)
We found a new solution (Total:19)
Finished after 4115 loops


<img src="dumbmonodromy.png" width="500">

In [13]:
#All of this is implemented in HomotopyContinuation.jl in a smarter way following Duff et.al.

MonSolve=monodromy_solve(F,S,C; show_progress=true)

MonodromyResult
• return_code → :heuristic_stop
• 19 solutions
• 228 tracked loops
• random_seed → 0xf4549049

<img src="smartermonodromy.png" width="500">

Under the assumption that random monodromy loops sample randomly from the set of possible permutations (which is completely not the case), as $\# solutions \to \infty$ you expect to only need two loops.

Let's check that we got the same answers

In [14]:
sort(solutions(MonSolve);lt=(x,y)->abs(real(x[1]))<abs(real(y[1])))

19-element Array{Array{Complex{Float64},1},1}:
 [-0.1110328323963439 - 0.901303004926522im]
 [0.15755124127140735 + 0.9871303370403146im]
 [-0.1943648926064032 + 0.6317701112189877im]
 [-0.2549371304226953 - 1.03379297378494im]
 [0.34947884523782413 - 0.9442874888326988im]
 [0.41677816104791265 + 0.9261127501202253im]
 [-0.4276930702027745 + 0.829879796461804im]
 [0.624463342550689 + 0.35052237621886345im]
 [0.7013718047647444 - 0.919029510780225im]
 [0.7087359677690267 - 0.3737800940437113im]
 [-0.7679106199345467 - 0.8671702466058969im]
 [0.8044682462062381 + 0.7799677662558236im]
 [-0.8380628626072111 + 0.7644498306770194im]
 [-0.8678529595611992 - 0.4151224877042829im]
 [0.923 + 0.19999999999999998im]
 [0.9947356196139029 - 0.31257396232446644im]
 [-1.0646731383858952 + 0.07701072074828227im]
 [-1.074517329689185 + 0.3920909900027887im]
 [-2.4849310703859375 - 0.46037859329255004im]

In [15]:
sort(SolutionsFound;lt=(x,y)->abs(real(x[1]))<abs(real(y[1])))

19-element Array{Array{Complex{Float64},1},1}:
 [-0.11103283239634384 - 0.9013030049265219im]
 [0.15755124127140735 + 0.9871303370403146im]
 [-0.1943648926064032 + 0.6317701112189877im]
 [-0.2549371304226953 - 1.03379297378494im]
 [0.34947884523782413 - 0.9442874888326988im]
 [0.41677816104791265 + 0.9261127501202253im]
 [-0.4276930702027745 + 0.829879796461804im]
 [0.624463342550689 + 0.35052237621886345im]
 [0.7013718047647444 - 0.919029510780225im]
 [0.7087359677690267 - 0.3737800940437114im]
 [-0.7679106199345467 - 0.8671702466058969im]
 [0.8044682462062381 + 0.7799677662558236im]
 [-0.8380628626072111 + 0.7644498306770194im]
 [-0.8678529595611992 - 0.4151224877042829im]
 [0.923 + 0.2im]
 [0.9947356196139029 - 0.31257396232446644im]
 [-1.064673138385895 + 0.07701072074828229im]
 [-1.074517329689185 + 0.3920909900027887im]
 [-2.4849310703859375 - 0.4603785932925501im]

## Monodromy Group

Let $\pi:Z|_U \to U$ be a $d$-to-one branched cover and $S_d$ the symmetric group acting on the points $\pi^{-1}(p)$ for  some $p \in U$.


**Definition**: A loop $\gamma$ based at some point $p \in U$ induces a permutation $g_\gamma \in S_d$ on the fibre $\pi^{-1}(p)$ via homotopy continuation.

**Definition**: The monodromy group $G_\pi$ of $\pi$ is the group of permutations $$G_\pi = \{g_\gamma \in S_d | \gamma \text{ is a loop in }U \text{ based at } p\}$$

**Exercise**: This doesn't depend on the base point $p \in U$

In [16]:
#This function will produce the permutation on the fibre BaseSols (over BaseParam) induced by the loop γ
#  which is just a sequence of parameter values starting and ending with BaseParam
#  If no such loop is given, it will randomly make one


function monodromy_element(F,BaseSols,BaseParam;γ=nothing)
    #If no loop is given, randomly make one
    if γ==nothing
        γ = [BaseParam] #starting at BaseParam
        for i in 1:3
            push!(γ,2*randn(ComplexF64,length(F.parameters))) #with 3 other parameter values
        end
        push!(γ,BaseParam) #and ending again at BaseParam
    end
    M = solutions(monodromy_loop(F,BaseSols,γ)) #Take the solutions after the loop
    S = (BaseSols) #And those before
    gᵧ= [findfirst(x->x≈S[i],M) for i in 1:length(M)] #and see which solutions were permuted
    return gᵧ
end

monodromy_element (generic function with 1 method)

In [17]:
@var x a b
F = System([x^2 + a * x + b], parameters = [a, b])
#Here's the monodromy element from before written as a permutation
monodromy_element(F,[[-1],[1]],p₀;γ=[p₀,p₁,p₂,p₃,p₀])

2-element Array{Int64,1}:
 2
 1

In [18]:
#here are 1000 random monodromy elements based at p_0
random_elements=[monodromy_element(F,[[-1],[1]],Array{ComplexF64,1}(p₀)) for i in 1:1000];

In [19]:
#this is a quick function to tally them
function tally(S)
    D=Dict{Any,Int}()
    for s in S
        D[s]=get(D,s,0)+1
    end
    return D
end

tally (generic function with 1 method)

In [20]:
#here's the tally
tally(random_elements)

Dict{Any,Int64} with 2 entries:
  [2, 1] => 345
  [1, 2] => 655

Take $$f(x,y) = y^4-2y^2-x,\\ g(x,y)=x-t$$


<img src="monondromy2.png" width="500">

In [21]:
@var x,y,t
G=System([y^4-2*y^2-x,x-t],parameters=[t])

System of length 2
 2 variables: x, y
 1 parameters: t

 -x - 2*y^2 + y^4
 -t + x

In [22]:
p₀ = [-1/2]
S=solutions(solve(G,target_parameters=p₀))

Tracking 4 paths... 100%|███████████████████████████████| Time: 0:00:03
  # paths tracked:                  4
  # non-singular solutions (real):  4 (4)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         4 (4)


4-element Array{Array{Complex{Float64},1},1}:
 [-0.5 - 3.587324068671532e-43im, 1.3065629648763766 + 8.96831017167883e-44im]
 [-0.5 + 0.0im, 0.541196100146197 + 0.0im]
 [-0.5 - 3.587324068671532e-43im, -1.3065629648763766 - 8.96831017167883e-44im]
 [-0.5 + 2.465190328815662e-32im, -0.541196100146197 + 0.0im]

In [23]:
random_elements=[monodromy_element(G,S,Array{ComplexF64,1}(p₀)) for i in 1:1000];

Tracking 4 paths... 100%|███████████████████████████████| Time: 0:00:01
  # paths tracked:                  4
  # non-singular solutions (real):  4 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         4 (0)


In [24]:
tally(random_elements)

Dict{Any,Int64} with 5 entries:
  [2, 1, 4, 3] => 106
  [1, 4, 3, 2] => 352
  [1, 2, 3, 4] => 521
  [4, 1, 2, 3] => 9
  [2, 3, 4, 1] => 12

In [25]:
using GAP;
function describe_group(L)
    L=unique(L)
    Sym=GAP.Globals.SymmetricGroup(length(L[1]))
    GAPL=[]
    for l in L
        push!(GAPL,GAP.Globals.PermList(GAP.julia_to_gap(l)))
    end
    GAPL=GAP.julia_to_gap(GAPL)
    G=GAP.Globals.Subgroup(Sym,GAPL)
    return (GAP.Globals.StructureDescription(G),G)
end

 ┌───────┐   GAP 4.11.0 of 29-Feb-2020
 │  GAP  │   https://www.gap-system.org
 └───────┘   Architecture: x86_64-pc-linux-gnu-julia64-kv7-v1.5
 Configuration:  gmp 6.1.2, Julia GC, Julia 1.5.2, readline
 Loading the library and packages ...
 Packages:   AClib 1.3.2, Alnuth 3.1.2, AtlasRep 2.1.0, AutoDoc 2019.09.04, 
             AutPGrp 1.10.2, CRISP 1.4.5, Cryst 4.1.23, CrystCat 1.1.9, 
             CTblLib 1.2.2, FactInt 1.6.3, FGA 1.4.0, Forms 1.2.5, 
             GAPDoc 1.6.3, genss 1.6.6, IO 4.7.0, IRREDSOL 1.4, LAGUNA 3.9.3, 
             orb 4.8.3, Polenta 1.3.9, Polycyclic 2.15.1, PrimGrp 3.4.0, 
             RadiRoot 2.8, recog 1.3.2, ResClasses 4.7.2, SmallGrp 1.4.1, 
             Sophus 1.24, SpinSym 1.5.2, TomLib 1.2.9, TransGrp 2.0.5, 
             utils 0.69
 Try '??help' for help. See also '?copyright', '?cite' and '?authors'


describe_group (generic function with 1 method)

In [26]:
describe_group(random_elements)

(GAP: "D8", GAP: D8)

**Observation**: There is a surjective map from the fundamental group of $U$ to the monodromy group. 

So given generators $\gamma_1,\ldots,\gamma_k$ of the fundamental group of $U$, the permutations they give in the monodromy group must generate the monodromy group.

**Example**: here the branch locus is $t=-1$ and $t=0$. Loops around $t=-1$ and $t=0$ should generate $G_\pi$
<img src="monondromy2.png" width="500">

In [27]:
g₁ = monodromy_element(G,S,Array{ComplexF64,1}(p₀),γ=[p₀,[-1+im],[-1.2],[-1-im],p₀])

4-element Array{Int64,1}:
 2
 1
 4
 3

In [28]:
g₂ = monodromy_element(G,S,Array{ComplexF64,1}(p₀),γ=[p₀,[0+im],[0.2],[0-im],p₀])

4-element Array{Int64,1}:
 1
 4
 3
 2

In [29]:
Gal=describe_group([g₁,g₂])
GAP.Globals.Order(Gal[2])

8

More generally: If $\ell$ is a general line in $Q$ (intersects $D$ at $\deg(D)$ many points), then the monodromy group $G_\pi$ is generated by the images of the loops around $D \cap \ell$ within $\ell$. 

**Exercise**: The monodromy group $G_\pi$ is transitive if and only if $Z$ is an irreducible variety.

**Exercise**: The monodromy group $G_\pi$ is $k$-transitive if and only if $\underline{\,\,\,\,\,\,\,\,\,\,\,\,}$ is an irreducible variety.

**Consequence**: If $Z$ is irreducible, and $s \in \pi^{-1}(p)$ then we can use monodromy to find **all** solutions. Otherwise we will fail at finding all solutions.

In [30]:
#Put it all together and compute monodromy group of 27 lines on cubic surface

**Cayley and Salmon**: There are exactly $27$ lines on every smooth cubic surface in $\mathbb{P}^3$.

Let's solve for these $27$ lines using monodromy, then compute the monodromy group!

*Source: Greg Egan* 

<img src="clebsch_diagonal_cubic-1-1.gif" width="500">
(The Clebsch cubic, a cubic surface with all 27 lines real; it is also special, as it has Eckhardt points)

In [31]:
# lines are parametrized as t->(t,k1+k3t,k2+k4t)
@var k[1:4]
# the coefficient of x^iy^jz^k in a general cubic will be c[i,j,k]
@var c[0:3,0:3,0:3]
# we need these variables too
@var t
@var x,y,z

#here's a cubic
cubic=sum([c[i+1,j+1,k+1]x^i*y^j*z^k for i in 0:3 for j in 0:3-i for k in 0:3-i-j])
#here's our line
line = [t,k[1]+k[3]*t,k[2]+k[4]*t]
#E is cubic(line(t))
E=evaluate(cubic,[x,y,z]=>line)
#and we construct the system where the coefficients of t^i for t=0..3 are zero (i.e. E=0)
F = System(exponents_coefficients(E,[t])[2],parameters=vec(c))

System of length 4
 4 variables: k₁, k₂, k₃, k₄
 64 parameters: c₀₋₀₋₀, c₁₋₀₋₀, c₂₋₀₋₀, c₃₋₀₋₀, c₀₋₁₋₀, c₁₋₁₋₀, c₂₋₁₋₀, c₃₋₁₋₀, c₀₋₂₋₀, c₁₋₂₋₀, c₂₋₂₋₀, c₃₋₂₋₀, c₀₋₃₋₀, c₁₋₃₋₀, c₂₋₃₋₀, c₃₋₃₋₀, c₀₋₀₋₁, c₁₋₀₋₁, c₂₋₀₋₁, c₃₋₀₋₁, c₀₋₁₋₁, c₁₋₁₋₁, c₂₋₁₋₁, c₃₋₁₋₁, c₀₋₂₋₁, c₁₋₂₋₁, c₂₋₂₋₁, c₃₋₂₋₁, c₀₋₃₋₁, c₁₋₃₋₁, c₂₋₃₋₁, c₃₋₃₋₁, c₀₋₀₋₂, c₁₋₀₋₂, c₂₋₀₋₂, c₃₋₀₋₂, c₀₋₁₋₂, c₁₋₁₋₂, c₂₋₁₋₂, c₃₋₁₋₂, c₀₋₂₋₂, c₁₋₂₋₂, c₂₋₂₋₂, c₃₋₂₋₂, c₀₋₃₋₂, c₁₋₃₋₂, c₂₋₃₋₂, c₃₋₃₋₂, c₀₋₀₋₃, c₁₋₀₋₃, c₂₋₀₋₃, c₃₋₀₋₃, c₀₋₁₋₃, c₁₋₁₋₃, c₂₋₁₋₃, c₃₋₁₋₃, c₀₋₂₋₃, c₁₋₂₋₃, c₂₋₂₋₃, c₃₋₂₋₃, c₀₋₃₋₃, c₁₋₃₋₃, c₂₋₃₋₃, c₃₋₃₋₃

 c₃₋₀₋₀ + k₃*c₂₋₁₋₀ + k₃^2*c₁₋₂₋₀ + k₃^3*c₀₋₃₋₀ + k₄*c₂₋₀₋₁ + k₄^2*c₁₋₀₋₂ + k₄^3*c₀₋₀₋₃ + k₄*k₃*c₁₋₁₋₁ + k₄*k₃^2*c₀₋₂₋₁ + k₄^2*k₃*c₀₋₁₋₂
 c₂₋₀₋₀ + k₁*c₂₋₁₋₀ + k₂*c₂₋₀₋₁ + k₃*c₁₋₁₋₀ + k₃^2*c₀₋₂₋₀ + k₄*c₁₋₀₋₁ + k₄^2*c₀₋₀₋₂ + 2*k₃*k₁*c₁₋₂₋₀ + k₃*k₂*c₁₋₁₋₁ + 3*k₃^2*k₁*c₀₋₃₋₀ + k₃^2*k₂*c₀₋₂₋₁ + k₄*k₁*c₁₋₁₋₁ + 2*k₄*k₂*c₁₋₀₋₂ + k₄*k₃*c₀₋₁₋₁ + k₄^2*k₁*c₀₋₁₋₂ + 3*k₄^2*k₂*c₀₋₀₋₃ + 2*k₄*k₃*k₁*c₀₋₂₋₁ + 2*k₄*k₃*k₂*c₀₋₁₋₂
 c₁₋₀₋₀ + k₁

### Great! Now solutions are parametrized lines 
$$ t \xrightarrow{\ell} (t,k1+k3t,k2+k4t) $$

such that 

$$cubic(\ell) = 0$$

In [32]:
@time MS=monodromy_solve(F)

  7.365505 seconds (16.20 M allocations: 819.006 MiB, 2.20% gc time)


MonodromyResult
• return_code → :heuristic_stop
• 27 solutions
• 189 tracked loops
• random_seed → 0x28407f00

Monodromy quickly solved the system!

In [33]:
#Output of monodromy_solve knows a fibre pi^(-1)(P) = S
S=solutions(MS);
P=parameters(MS);

In [34]:
@time random_elements=[monodromy_element(F,S,P) for i in 1:50];

  2.765097 seconds (6.38 M allocations: 312.349 MiB, 3.57% gc time)


In [35]:
#Here we filter out anything funny that happened numerically (note we weren't particularly
#  careful with our code earlier)
filter!(x->!(nothing in x) && sort(x) == collect(1:27),random_elements);
length(random_elements)

47

In [36]:
tally(unique(random_elements))

Dict{Any,Int64} with 46 entries:
  [26, 14, 3, 5, 4, 21, 25, 1, 15, 8  …  7, 19, 11, 6, 2, 9, 17, 23, 16, 2… => 1
  [22, 21, 12, 9, 5, 3, 6, 24, 18, 8  …  13, 15, 16, 25, 26, 1, 7, 17, 23,… => 1
  [18, 4, 19, 9, 7, 1, 14, 25, 11, 17  …  21, 13, 5, 12, 16, 6, 20, 22, 24… => 1
  [18, 4, 26, 1, 24, 25, 13, 8, 10, 19  …  2, 15, 20, 3, 12, 23, 22, 6, 14… => 1
  [15, 14, 2, 22, 17, 18, 19, 6, 20, 13  …  5, 23, 4, 3, 27, 25, 16, 12, 1… => 1
  [18, 24, 15, 8, 17, 13, 27, 19, 12, 9  …  26, 4, 6, 5, 23, 11, 16, 3, 1,… => 1
  [9, 15, 8, 24, 4, 10, 11, 18, 7, 19  …  13, 25, 3, 14, 1, 16, 27, 20, 21… => 1
  [20, 21, 26, 14, 16, 12, 23, 27, 2, 3  …  13, 6, 19, 18, 4, 10, 17, 15, … => 1
  [27, 21, 4, 3, 17, 25, 7, 2, 22, 11  …  14, 6, 19, 8, 9, 23, 15, 5, 1, 2… => 1
  [16, 4, 3, 1, 20, 12, 23, 27, 18, 26  …  17, 7, 22, 2, 21, 24, 13, 15, 6… => 1
  [1, 19, 18, 26, 2, 15, 6, 4, 12, 8  …  16, 3, 13, 25, 9, 22, 11, 20, 27,… => 1
  [1, 17, 4, 19, 3, 27, 14, 10, 23, 16  …  15, 5, 21, 9, 11, 20, 22, 24, 8… 

 So we may have lost some permutations, but we still have plenty, and since there aren't any duplicates (seemingly), the group is probably pretty big

In [37]:
Gal=describe_group(random_elements)
println(Gal[1])
GAP.Globals.Order(Gal[2])

GAP: "O(5,3) : C2"


51840

##### This is the Weyl group of E6

It is exactly the group of permutations which are possible given how the $27$ lines must intersect.

General philosophy: the monodromy group is always as big as possible given the geometric obstructions (not actually a mathematical statement)

## Conclusion
- we can solve systems using monodromy provided we know the incidence variety is irreducible
 - How do we know when to stop? (stay tuned for witness sets talk)
- we can compute monodromy groups which reveal structure about our system
 - Find your favorite family of zero-dimensional varieties and compute their monodromy groups
   - Schubert Problems
   - Sparse polynomial systems
   - Computer vision